In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import random

In [2]:
# Load dataset
df = pd.read_csv("../datasets/loan_predication/loan_predication.csv")

df = df.drop(columns=['Loan_ID'])
# Check missing values
print("Missing values per column:\n")
print(df.isnull().sum())

# Encode target with LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['Loan_Status'])
X = df.drop(columns=['Loan_Status'])

# Apply LabelEncoder on all non-numeric columns
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# Handle missing values (median for numeric, mode for categorical)
X = X.fillna(X.median(numeric_only=True))
X = X.fillna(X.mode().iloc[0])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape

Missing values per column:

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


((429, 11), (185, 11))

In [4]:
X_test.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
350,1,1,0,0,0,9083,0.0,228.0,360.0,1.0,1


In [15]:
# Define states from dataset (Credit_History + ApplicantIncome)
def get_state(row):
    if row['Credit_History'] ==0:
        return 2 # High risk
    elif row['ApplicantIncome'] > df['ApplicantIncome'].median():
        return 1 # medium risk
    else:
        return 0 # low risk

states = ['low_risk','medium_risk','high_risk']
actions = ['ignore','review','escalate']

q_table = np.zeros((len(states),len(actions)))
alpha, gamma, episodes = 0.1,0.9,500

# Training Loop
for _ in range(episodes):
    idx = random.choice(X_test.index)
    row = df.loc[idx]
    state = get_state(row)
    action = random.choice(range(len(actions)))

    # Reward policy:
    if state == 2 and action == 2:    #escalate high_risk
        reward = 5
    elif state == 0 and action == 0:  # ignore low_risk
        reward = 3
    elif state == 1 and action == 1:   # review medium_risk
        reward = 2
    else:
        reward = -2                    #penalize if wrong handling
    
    best_next = np.max(q_table[state])
    q_table[state,action] += alpha * (reward + gamma * best_next - q_table[state,action])

q_df = pd.DataFrame(q_table, columns=actions,index=states)

print('Q-table learned for banking loan alert prioritization')
q_df.round(2)

Q-table learned for banking loan alert prioritization


,ignore,review,escalate
low_risk,16.44,10.41,11.35
medium_risk,5.44,9.90,5.32
high_risk,4.15,5.45,10.32
